<a href="https://colab.research.google.com/github/ssvadla/Bridge_Project/blob/main/Supervised_CUS_Light_XG_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import time
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
train = pd.read_csv('/content/drive/My Drive/Bridge_Project/ConterminousUS/ConterminousUSPre.csv')
#train = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTrain.csv')

In [58]:
train.columns

Index(['Unnamed: 0', 'Skew', 'Number_Spans', 'Length_Max_Span',
       'Main_Design_Type', 'Deck_Design_Type', 'Wearing_Surface',
       'Deck_Protection', 'ADT', 'ADTT', 'State', 'Structure_Number',
       'Latitude', 'Longitude', 'Deck_Condition', 'Superstructure_Condition',
       'Substructure_Condition', 'Freq_Shake_GT4', 'Max_Shake',
       'Days_MinLT_0F', 'Days_MinLT_32F', 'Days_MaxGT_70F', 'Days_MaxGT_90F',
       'Avg_Wind_Speed', 'Peak_Wind_Speed', 'Precipitation', 'Temp_Max',
       'Temp_Mean', 'Temp_Min', 'Reconstructed', 'Age', 'Deck_Area'],
      dtype='object')

In [59]:
#Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude', 'Superstructure_Condition', 'Substructure_Condition']
#Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude']
Excluded_columns = ['Unnamed: 0','State', 'Structure_Number', 'Latitude', 'Longitude','Days_MinLT_0F','Days_MaxGT_90F','Avg_Wind_Speed','Peak_Wind_Speed','Peak_Wind_Speed','Precipitation']

In [60]:
train = train.drop(columns = Excluded_columns)

In [61]:
train.columns

Index(['Skew', 'Number_Spans', 'Length_Max_Span', 'Main_Design_Type',
       'Deck_Design_Type', 'Wearing_Surface', 'Deck_Protection', 'ADT', 'ADTT',
       'Deck_Condition', 'Superstructure_Condition', 'Substructure_Condition',
       'Freq_Shake_GT4', 'Max_Shake', 'Days_MinLT_32F', 'Days_MaxGT_70F',
       'Temp_Max', 'Temp_Mean', 'Temp_Min', 'Reconstructed', 'Age',
       'Deck_Area'],
      dtype='object')

In [63]:
train.shape

(409546, 22)

In [64]:
(np.unique(train['Superstructure_Condition'].values))

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'N'],
      dtype=object)

In [65]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['Main_Design_Type_Encoded'] = le.fit_transform(train['Main_Design_Type'])
train['Deck_Design_Type_Encoded'] = le.fit_transform(train['Deck_Design_Type'])
train['Wearing_Surface_Encoded'] = le.fit_transform(train['Wearing_Surface'])
train['Deck_Protection_Encoded'] = le.fit_transform(train['Deck_Protection'])
#train['Superstructure_Condition_Encoded'] = le.fit_transform(train['Superstructure_Condition'])


In [ ]:
 count = 0
 for value in train['Substructure_Condition']:
   #print(value)
   if value == 'N':
     print("Yes")
     count = count + 1

In [67]:
count

26562

In [68]:
indexNames = train[ train['Substructure_Condition'] == 'N' ].index

In [69]:
indexNames

Int64Index([    19,     28,     73,    114,    148,    181,    193,    220,
               223,    257,
            ...
            408879, 408892, 408900, 408920, 409055, 409118, 409149, 409208,
            409278, 409377],
           dtype='int64', length=26562)

In [70]:
train.shape

(409546, 26)

In [71]:
train.drop(indexNames , inplace=True)

In [72]:
train.shape

(382984, 26)

In [73]:
train['Substructure_Condition_typed']= pd.to_numeric(train['Substructure_Condition'])

In [74]:
np.unique(train['Substructure_Condition_typed'].values)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [75]:
 count = 0
 for value in train['Superstructure_Condition']:
   #print(value)
   if value == 'N':
     print("Yes")
     count = count + 1

Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes
Yes


In [76]:
count

19

In [77]:
indexNames_super = train[ train['Superstructure_Condition'] == 'N' ].index

In [78]:
train.drop(indexNames_super , inplace=True)

In [79]:
train.shape

(382965, 27)

In [80]:
train['Superstructure_Condition_typed']= pd.to_numeric(train['Superstructure_Condition'])

In [81]:
train['Substructure_Condition_Encoded'] = le.fit_transform(train['Substructure_Condition_typed'])
train['Superstructure_Condition_Encoded'] = le.fit_transform(train['Superstructure_Condition_typed'])

In [82]:
import lightgbm as lgb
classifier_lgb = lgb.LGBMClassifier()

In [83]:
X = train.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection','Substructure_Condition_typed','Substructure_Condition','Superstructure_Condition_typed','Superstructure_Condition']) 


In [84]:
y = train['Deck_Condition']

In [86]:
X.shape

(382965, 21)

In [87]:
X.columns

Index(['Skew', 'Number_Spans', 'Length_Max_Span', 'ADT', 'ADTT',
       'Freq_Shake_GT4', 'Max_Shake', 'Days_MinLT_32F', 'Days_MaxGT_70F',
       'Temp_Max', 'Temp_Mean', 'Temp_Min', 'Reconstructed', 'Age',
       'Deck_Area', 'Main_Design_Type_Encoded', 'Deck_Design_Type_Encoded',
       'Wearing_Surface_Encoded', 'Deck_Protection_Encoded',
       'Substructure_Condition_Encoded', 'Superstructure_Condition_Encoded'],
      dtype='object')

In [89]:
Extra_Excluded = ['Freq_Shake_GT4','Max_Shake','Days_MinLT_32F','Days_MaxGT_70F','Temp_Max','Temp_Mean','Temp_Min']

In [90]:
X = X.drop(columns = Extra_Excluded)

In [91]:
X.shape

(382965, 14)

In [92]:
from sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=42)


In [93]:
start = time.time()
classifier_lgb.fit(X_train, Y_train)
end = time.time()
print(f"Runtime of the program is {end - start}")

Runtime of the program is 37.138875246047974


In [40]:
# test = pd.read_csv('/content/drive/My Drive/Bridge_Project/Dataset_1/ConterminousUSPre5StatesKnownTest.csv')

In [41]:
# test = test.drop(columns = Excluded_columns)

In [42]:
# test['Main_Design_Type_Encoded'] = le.fit_transform(test['Main_Design_Type'])
# test['Deck_Design_Type_Encoded'] = le.fit_transform(test['Deck_Design_Type'])
# test['Wearing_Surface_Encoded'] = le.fit_transform(test['Wearing_Surface'])
# test['Deck_Protection_Encoded'] = le.fit_transform(test['Deck_Protection'])

In [43]:
# #train['Substructure_Condition_typed']= pd.to_numeric(train['Substructure_Condition'])
# test['Substructure_Condition_Encoded'] = le.fit_transform(test['Substructure_Condition'])

In [44]:
# X_test = test.drop(columns=['Deck_Condition','Main_Design_Type','Deck_Design_Type','Wearing_Surface','Deck_Protection','Substructure_Condition']) 


In [45]:
# y_test = test['Deck_Condition']

In [46]:
# np.unique(train['Deck_Condition'].values)

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'N'],
      dtype=object)

In [47]:
# np.unique(test['Deck_Condition'].values)

array([ 6,  7, 45, 89])

In [33]:
# X_test.shape

(13970, 14)

In [94]:
test_pred = classifier_lgb.predict(x_test)

In [95]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test,test_pred, digits=4)

In [96]:
print(classification_report)

              precision    recall  f1-score   support

           0     0.3100    0.8857    0.4593        35
           1     0.0000    0.0000    0.0000         5
           2     0.0435    0.3750    0.0779        24
           3     0.3904    0.2979    0.3379       329
           4     0.5060    0.3612    0.4215      1869
           5     0.5501    0.4639    0.5033      9005
           6     0.5960    0.5725    0.5840     19392
           7     0.6968    0.7680    0.7307     33765
           8     0.6735    0.5965    0.6326     10455
           9     0.7094    0.7520    0.7301      1633
           N     0.0407    0.0864    0.0553        81

    accuracy                         0.6462     76593
   macro avg     0.4106    0.4690    0.4121     76593
weighted avg     0.6440    0.6462    0.6431     76593



In [98]:
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
classifier_xg = XGBClassifier()
start = time.time()
classifier_xg.fit(X_train, Y_train)
end = time.time()
test_pred_xg = classifier_xg.predict(x_test)
classification_report_xg = classification_report(y_test,test_pred_xg, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_xg)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Runtime of the program is 198.90410709381104
              precision    recall  f1-score   support

           0     0.9118    0.8857    0.8986        35
           1     0.0000    0.0000    0.0000         5
           2     0.6296    0.7083    0.6667        24
           3     0.6080    0.3678    0.4583       329
           4     0.5414    0.4056    0.4638      1869
           5     0.5545    0.4830    0.5163      9005
           6     0.6180    0.5743    0.5953     19392
           7     0.7019    0.7881    0.7425     33765
           8     0.6989    0.6242    0.6594     10455
           9     0.7237    0.8053    0.7623      1633
           N     0.0000    0.0000    0.0000        81

    accuracy                         0.6641     76593
   macro avg     0.5444    0.5129    0.5239     76593
weighted avg     0.6583    0.6641    0.6589     76593



In [99]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(n_estimators = 350, criterion = 'gini', max_features = 'auto', random_state = 42)
start = time.time()
classifier_rf.fit(X_train, Y_train)
end = time.time()
test_pred_rf = classifier_rf.predict(x_test)
classification_report_rf = classification_report(y_test,test_pred_rf, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_rf)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Runtime of the program is 263.2430157661438
              precision    recall  f1-score   support

           0     0.9688    0.8857    0.9254        35
           1     0.0000    0.0000    0.0000         5
           2     1.0000    0.1250    0.2222        24
           3     0.5987    0.2766    0.3784       329
           4     0.5953    0.3943    0.4744      1869
           5     0.6052    0.5260    0.5629      9005
           6     0.6399    0.6170    0.6282     19392
           7     0.7295    0.8175    0.7710     33765
           8     0.7427    0.6528    0.6948     10455
           9     0.7928    0.7826    0.7877      1633
           N     1.0000    0.0123    0.0244        81

    accuracy                         0.6955     76593
   macro avg     0.6975    0.4627    0.4972     76593
weighted avg     0.6919    0.6955    0.6905     76593



In [ ]:
from sklearn import svm
classifier_svm = svm.SVC(decision_function_shape='ovo')
start = time.time()
classifier_svm.fit(X_train, Y_train)
end = time.time()
test_pred_svm = classifier_svm.predict(x_test)
classification_report_svm = classification_report(y_test,test_pred_svm, digits=4)
print(f"Runtime of the program is {end - start}")
print(classification_report_svm)